In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests

In [ ]:
fre_mulitplier = 78.5

countries = ['bangladesh', 'brazil', 'cambodia', 'china', 'indonesia', 'mexico','rwanda','kenya']

city_country = ['brazil', 'cambodia','rwanda','kenya']

def convert_to_flt(str):
    try:
        return float(str.replace(",", ""))  
    except ValueError:
        return None

In [84]:
def make_df_admin(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    rows = soup.find_all('tr', class_=['rname', 'admin2'])
    # rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')
    # import pdb; pdb.set_trace()
    years = []
    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        # import pdb; pdb.set_trace()
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            years.append(year)


        
    for row in rows:
        try:
            # Get the name, status, and population for each census year
            name = row.find('td', class_='rname').text.strip()
            status = row.find('td', class_='rstatus').text.strip()

            # Extract area and population data for different census years
            # import pdb; pdb.set_trace()
            area = convert_to_flt(row.find('td', class_='rname')['data-area'])  # Get the 'data-area' attribute

            # Extract all population data (there could be multiple columns for different years)
            population_cells = row.find_all('td', class_='rpop')

            # Create a dictionary to store population data for each year
            population_data = {}
            for i, cell in enumerate(population_cells):
                population = cell.text.strip()
                population_data[f'Population {years[i]}'] = convert_to_flt(population)

            # Append the extracted data into the list
            row_data = [name, status, area] + list(population_data.values())
            data.append(row_data)
            
        except:
            continue
    # Create a DataFrame
    columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column

In [85]:
def make_df_city(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    # rows = soup.find_all('tr', class_=['rname', 'admin2'])
    rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')

    years = []

    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            if year not in years:
                years.append(year)

        
    for row in rows:
        try:
            # Get the name, status, and population for each census year
            name = row.find('td', class_='rname').text.strip()
            status = row.find('td', class_='rstatus').text.strip()

            # Extract area and population data for different census years
            # import pdb; pdb.set_trace()
            area = row.find('td', class_='rarea').text.strip()  # Get the 'data-area' attribute

            # Extract all population data (there could be multiple columns for different years)
            population_cells = row.find_all('td', class_='rpop')

            # Create a dictionary to store population data for each year
            population_data = {}
            for i, cell in enumerate(population_cells):
                population = cell.text.strip()
                population_data[f'Population {years[i]}'] = convert_to_flt(population)

            # Append the extracted data into the list
            row_data = [name, status, area] + list(population_data.values())
            data.append(row_data)
            
        except Exception as e:
            import pdb; pdb.set_trace()
            continue
    # Create a DataFrame
    columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column

In [86]:
def make_df_city(page, section_id):
    soup = BeautifulSoup(page.content, 'html.parser')
    soup = soup.find(id=section_id)
    # print(soup.prettify())

    # rows = soup.find_all('tr', class_=['rname', 'admin2'])
    rows = soup.find_all('tr')

    data = []

    header_cells = soup.find_all('th', class_='rpop')

    years = []

    for header in header_cells:
        # Extract the year from the 'span' element inside the 'th' (e.g., "2005-06-30")
        span = header.find('span', class_='unit')
        if span:
            # Extract year from the text, assuming it's in the form "Census\nYYYY-MM-DD"
            year = span.text.strip().split('-')[0][-4:]  # Get the year part
            if year not in years:
                years.append(year)

    # import pdb; pdb.set_trace()
        
    for row in rows:
        if section_id == 'adminareas':
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', class_='rname').text.strip()
                status = row.find('td', class_='rstatus').text.strip()

                # Extract area and population data for different census years
                # import pdb; pdb.set_trace()
                area = row.find('td', class_='rarea').text.strip()  # Get the 'data-area' attribute

                # Extract all population data (there could be multiple columns for different years)
                population_cells = row.find_all('td', class_='rpop')

                # Create a dictionary to store population data for each year
                population_data = {}
                for i, cell in enumerate(population_cells):
                    population = cell.text.strip()
                    population_data[f'Population {years[i]}'] = convert_to_flt(population)

                # Append the extracted data into the list
                row_data = [name, status, area] + list(population_data.values())
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame
          

        elif section_id == "largecities":
            # import pdb; pdb.set_trace()
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', {'class': None}).text.strip()
                # Extract all population data (there could be multiple columns for different years)
                population_tag = row.find('td', {'class': 'rpop'}).text.strip()

                # Append the extracted data into the list
                row_data = [name, population_tag]
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame

        elif section_id == "citysection":
            # import pdb; pdb.set_trace()
            try:
                # Get the name, status, and population for each census year
                name = row.find('td', class_='rname').text.strip()
                # status = row.find('td', class_='rstatus').text.strip()

                # Extract area and population data for different census years
                # import pdb; pdb.set_trace()
                area = row.find('td', class_='noviz').text.strip()  # Get the 'data-area' attribute

                # Extract all population data (there could be multiple columns for different years)
                population_cells = row.find_all('td', class_='rpop')

                # Create a dictionary to store population data for each year
                population_data = {}
                for i, cell in enumerate(population_cells):
                    population = cell.text.strip()
                    population_data[f'Population {years[i]}'] = convert_to_flt(population)

                # Append the extracted data into the list
                row_data = [name, area] + list(population_data.values())
                data.append(row_data)
                
            except Exception as e:
                # import pdb; pdb.set_trace()
                continue
            # Create a DataFrame

        
    
    if section_id == 'adminareas':
        columns = ['Name', 'Status', 'Area'] + [year for year in years]  
    elif section_id == 'largecities':
        columns = ['Name'] + [year for year in years] 
    elif section_id == 'citysection':
        columns = ['Name', 'Area'] + [year for year in years] 

    df = pd.DataFrame(data, columns=columns)

    latest_population_column = years[-1]

    return df, latest_population_column



In [87]:
test, year = make_df_city(page, 'citysection')
test

Exception ignored in: <function ZipFile.__del__ at 0x0000020570DFCF40>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile.py", line 1894, in __del__
    self.close()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile.py", line 1911, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
Exception ignored in: <function ZipFile.__del__ at 0x0000020570DFCF40>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile.py", line 1894, in __del__
    self.close()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile.py", line 1911, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
page = requests.get('https://www.citypopulation.de/en/brazil/cities/')
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())
# df, latest_population_column = make_df_admin(page)


<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="Brazil: States and Major Cities with population statistics, maps, charts, weather and web information." name="description"/>
  <title>
   Brazil: States and Major Cities - Population Statistics, Maps, Charts, Weather and Web Information
  </title>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <script src="/js/cp_data_m.js">
  </script>
  <script src="/js/cp_citypage_m.js">
  </script>
  <script src="/js/cp_menu.js">
  </script>
  <style>
   article.cpage { top: 469px }
    div#map { height: 369px }
    @media screen and (min-width: 1395px) {
      article.cpage { top: 99px; left: 720px }
      div#map { left: 160px; top: -1px; height: auto; bottom: -1px; width: 540px }
      header.cpage { left: 720px }
      div#headline { left: 702px }
    }
  </style>
 </head>
 <body itemscope="" itemtype="http://schema.org/Country" onload="start()">
  <script>
   writeMenu('en','htmlpage','brazil-cities')
  </

In [ ]:
tables = ['adminareas', 'largecities', "citysection"]

In [ ]:
for country in countries:
    print(country)
    if country not in city_country:
        page = requests.get(f"https://www.citypopulation.de/en/{country}/admin/")
        df, year = make_df_admin(page)

    
    else:
        page = requests.get(f"https://www.citypopulation.de/en/{country}/cities/")
        admin_df, year = make_df_city(page, 'adminareas')
        city_df, year = make_df_city(page, 'largecities')
        city_section_df, year = make_df_city(page, 'citysection')

        df = pd.concat([admin_df, city_df, city_section_df], axis=0)

        # country_admin_df_name = f"{country}_admin_df"
        # country_city_df_name = f"{country}_city_df"
        # country_city_section_df_name = f"{country}_city_section_df"
        # globals()[country_admin_df_name] = admin_df
        # globals()[country_city_df_name] = city_df
        # globals()[country_city_section_df_name] = city_section_df

# import pdb; pdb.set_trace()
        country_df_name = f"{country}_df"  
        country_year_name = f"{country}_year"
        globals()[country_df_name] = df
        globals()[country_year_name] = year
    
    


bangladesh
brazil
cambodia
china
indonesia
mexico


In [88]:
with pd.ExcelWriter('citypopulation_data.xlsx', engine='openpyxl') as writer:
    bangladesh_df.to_excel(writer, sheet_name='Bangladesh', index=False)
    brazil_df.to_excel(writer, sheet_name='Brazil', index=False)
    cambodia_df.to_excel(writer, sheet_name='Cambodia', index=False)
    china_df.to_excel(writer, sheet_name='China', index=False)
    indonesia_df.to_excel(writer, sheet_name='Indonesia', index=False)
    mexico_df.to_excel(writer, sheet_name='Mexico', index=False)
    workbook = writer.book
    for sheet in workbook.sheetnames:
        workbook[sheet].sheet_state = 'visible'
    writer.save()


IndexError: At least one sheet must be visible